In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [2]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run, run_get_node
from aiida.engine import run_get_node
from aiida.orm import load_code, load_node
from alamode_aiida.ase_support import load_atoms
from ase import io
from tools import wait_for_node_finished

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')
TrajectoryData = DataFactory('array.trajectory')


In [3]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os

home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
print(computer_local)
code = get_or_create_code('alamode.displace_pf',
                          computer_local,
                          '/home/max/Documents/alamode/tools/displace.py')
code_displace_pf = code
code = get_or_create_code('alamode.displace_random',
                          computer_local,
                          '/home/max/Documents/alamode/tools/displace.py')
code_displace_random = code
code = get_or_create_code('alamode.lammps',
                         computer_local,
                         '/usr/bin/lammps')
code_lammps_name = Str('alamode.lammps@localhost')

code = get_or_create_code('alamode.extract',
                         computer_local,
                         '/home/max/Documents/alamode/tools/extract.py')
code_extract = code
code_extract_name = Str('alamode.extract@localhost')


code_displace_random, code_displace_pf, code_lammps_name


localhost (localhost), pk: 5


(<Code: Remote code 'alamode.displace_random' on localhost, pk: 59133, uuid: 9ac4bf3b-061b-4b54-b71c-720c1629204c>,
 <Code: Remote code 'alamode.displace_pf' on localhost, pk: 59132, uuid: 1edf312a-d038-4843-b752-6673dfabb07b>,
 <Str: uuid: c0623040-8014-4597-bdaf-f2037cf43603 (unstored) value: alamode.lammps@localhost>)

In [4]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


{'CWD': 'run29'}


'/home/max/Documents/aiida-test/alamode-aiida/example/run29'

In [5]:

from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [6]:
g_cwd = nodebank.load("cwd")  # ファイル保存directory
g_superstructure = nodebank.load("superstructure")  # 使用する長周期構造StructureData
g_superstructure_file = nodebank.load(
    "superstructure_file")  # 使用する長周期構造ファイルSinglefileData
g_superstructure_filepath = nodebank.load(
    "superstructure_filepath")  # 使用する長周期構造ファイルabspath
g_superstructure_filename = nodebank.load(
    "superstructure_filename")  # 使用する長周期構造ファイルfilename in the cwd directory

g_file_format = nodebank.load("structure_org_format")  # lammpsの位置フォーマット
g_file_style = nodebank.load("structure_org_style")  # lammpsの位置フォーマットのオプション
g_file_format, g_file_style, g_superstructure_file


(<Str: uuid: 1cc62608-2e6e-45ba-8f5c-b2e355fd4ae1 (pk: 59096) value: lammps-data>,
 <Str: uuid: 99a37364-9e4b-4e0a-aa55-0fad6b788081 (pk: 59097) value: atomic>,
 <SinglefileData: uuid: 28128ea5-41f8-4dd5-a101-fb575732c817 (pk: 59117)>)

In [7]:
g_prefix = nodebank.load('prefix')
g_prefix

<Str: uuid: 2d95abd8-df96-46cf-a41e-921191bdf045 (pk: 59120) value: harmonic>

## for LAMMPS

In [8]:
key = "format"
g_format = nodebank.load_or_dump(key, Str("LAMMPS"))  # alamodeのファイルオプションの定義


debug, node uuid: d7a6f1a8-bf05-47af-9cff-6add217853e4 (unstored) value: LAMMPS
debug, load uuid: 74e46851-89aa-44b7-be04-1be4721a28df (pk: 59136) value: LAMMPS


In [9]:
from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version
computer_local = get_or_create_local_computer('work_directory', 'localhost')
code_lammps_force = get_or_create_code('lammps.force', computer_local, 'lammps')

In [10]:
code_lammps_force_name = Str(code_lammps_force.label+"@"+'localhost')
code_lammps_force_name

<Str: uuid: c8abc276-f999-4c57-8ffe-19095c1681b4 (unstored) value: lammps.force-lammps-localhost@localhost>

In [11]:
code_lammps_force.get_from_string(code_lammps_force_name.value)


<Code: Remote code 'lammps.force-lammps-localhost' on localhost, pk: 59137, uuid: 38d58d76-d1ac-419e-97e6-24a749d7c791>

In [12]:
# make lammps in and out

In [13]:
g_mag = nodebank.load_or_dump("mag", Float(0.01))


debug, node uuid: 9fa5e7aa-605f-422a-a691-cc7e5c78eae7 (unstored) value: 0.01
debug, load uuid: 9f9d275b-5ce7-457b-a9e2-56c1211b087f (pk: 59138) value: 0.01


In [14]:

displacement_patterns = nodebank.load(f'{g_prefix.value}_pattern_folder')
displacement_patterns


<FolderData: uuid: fca74e9f-4cd9-4fe8-8932-097f48666aa3 (pk: 59131)>

In [15]:
g_superstructure.get_ase()

Atoms(symbols='Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2', pbc=True, cell=[[6.38, 0.0, 0.0], [-3.19, 5.5252420761447, 0.0], [0.0, 0.0, 10.378]], masses=...)

In [16]:
g_superstructure_filename

<Str: uuid: c60d5c7e-a3c8-4b16-8707-f5fc9ed8a75b (pk: 59119) value: supercell.lammps>

In [17]:
displacement_patterns

<FolderData: uuid: fca74e9f-4cd9-4fe8-8932-097f48666aa3 (pk: 59131)>

In [18]:
TrajectoryData = DataFactory('array.trajectory')
from alamode_aiida.ase_support import load_atoms_bare


In [19]:
g_action = "pf"
pattern_filenames = List(list=[f'pattern.{g_prefix.value}'])

if g_action == "pf":

    #codename = "alamode.displace_pf@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_pf

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure
    builder.structure_org_filename = g_superstructure_filename
    builder.mag = g_mag
    builder.pattern_files = displacement_patterns
        
    g_displacefuture, node = run_get_node(builder)
    print("node",node)
    
    g_displacefuture = nodebank.load(f"{g_prefix.value}_displace_pf")
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump(f"{g_prefix.value}_displace_pf", g_displacefuture)

elif g_action == "random":

    g_num_disp = nodebank.load_or_dump(f"{g_prefix.value}_num_disp", Int(5))

    #codename = "alamode.displace_random@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_random

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure
    builder.structure_org_filename = g_superstructure_filename
    builder.mag = g_mag
    builder.num_disp = g_num_disp

    g_displacefuture = nodebank.load(f"{g_prefix.value}_displace_random")
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump(f"{g_prefix.value}_displace_random", g_displacefuture)
        else:
            raise
else:
    raise ValueError("unknown action")


prepare_for_submission done
node uuid: e1aeffe8-51d8-4375-9db7-6ce9a5b132eb (pk: 59184) (aiida.calculations:alamode.displace_pf)
uuid: 73414836-7a88-4568-bbdb-de9488d736c0 (pk: 59193) (aiida.calculations:alamode.displace_pf)
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
True True


In [20]:
g_displacefuture.is_finished_ok


True

In [21]:
g_displacefuture.attributes


{'job_id': '5870',
 'sealed': True,
 'version': {'core': '1.6.8'},
 'withmpi': False,
 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1},
 'append_text': '',
 'exit_status': 0,
 'parser_name': 'alamode.displace',
 'prepend_text': '',
 'last_job_info': {'job_id': '5870',
  'job_owner': 'max',
  'job_state': 'running',
  'wallclock_time_seconds': 0},
 'process_label': 'displace_pf_Calcjob',
 'process_state': 'finished',
 'retrieve_list': ['displace_pf.in',
  'displace_pf.out',
  'harmonic*.lammps',
  '_scheduler-stdout.txt',
  '_scheduler-stderr.txt'],
 'input_filename': 'displace_pf.in',
 'remote_workdir': '/scratch/max/aiida/73/41/4836-7a88-4568-bbdb-de9488d736c0',
 'output_filename': 'displace_pf.out',
 'scheduler_state': 'done',
 'scheduler_stderr': '_scheduler-stderr.txt',
 'scheduler_stdout': '_scheduler-stdout.txt',
 'detailed_job_info': None,
 'mpirun_extra_params': [],
 'environment_variables': {},
 'import_sys_environment': True,
 'submit_script_filename': '_aiida

In [22]:
g_displacefuture.outputs.retrieved.list_object_names()


['_scheduler-stderr.txt',
 '_scheduler-stdout.txt',
 'displace_pf.out',
 'harmonic01.lammps',
 'harmonic02.lammps',
 'harmonic03.lammps',
 'harmonic04.lammps',
 'harmonic05.lammps',
 'harmonic06.lammps',
 'harmonic07.lammps',
 'harmonic08.lammps',
 'harmonic09.lammps',
 'harmonic10.lammps']

In [23]:
for filename in g_displacefuture.outputs.retrieved.list_object_names():
    if filename.startswith("displace"):
        print(g_displacefuture.outputs.retrieved.get_object_content(filename))


*****************************************************************
    displace.py --  Generator of displaced configurations        
                      Version. 1.2.1                             
*****************************************************************

Namespace(mag=0.01, prefix='harmonic', QE=None, VASP=None, xTAPP=None, LAMMPS='supercell.lammps', OpenMX=None, pattern_file=['harmonic.pattern_HARMONIC'], random=False, random_normalcoord=False, temp=100, ignore_imag=False, every='50', load_mddata=None, prim=None, evec=None, num_disp=1, classical=False, print_disp_stdout=False, pes=None, imag_evec=False, Qrange=None)
<class 'argparse.Namespace'>
 Output format                  : LAMMPS structure format
 Structure before displacements : supercell.lammps
 Output file names              : harmonic{counter}.lammps
 Magnitude of displacements     : 0.01 Angstrom
 Number of atoms                : 32

deug dispobj <GenDisplacement.AlamodeDisplace object at 0x1500c2c5beb0>
 Displacem

In [24]:
g_displacefuture.outputs.dispfile_folder.list_object_names()


['harmonic01.lammps',
 'harmonic02.lammps',
 'harmonic03.lammps',
 'harmonic04.lammps',
 'harmonic05.lammps',
 'harmonic06.lammps',
 'harmonic07.lammps',
 'harmonic08.lammps',
 'harmonic09.lammps',
 'harmonic10.lammps']

In [25]:
g_displace_result = g_displacefuture.outputs.results
g_displace_result


<Dict: uuid: 24bcde8a-e358-402c-a37c-73233f03ed61 (pk: 59196)>

In [26]:
g_dispfile_folder = g_displacefuture.outputs.dispfile_folder


In [27]:
print(g_displacefuture.outputs.displaced_structures.get_stepids())
structure = g_displacefuture.outputs.displaced_structures.get_step_structure(0)
structure.get_ase().get_positions()


[0 1 2 3 4 5 6 7 8 9]


array([[ 1.00000000e-02,  1.84174736e+00,  5.20021239e+00],
       [ 1.59500000e+00,  9.20873679e-01,  2.60571239e+00],
       [ 2.20545804e-15,  1.84174736e+00,  1.96060761e+00],
       [ 1.59500000e+00,  9.20873679e-01,  4.55510761e+00],
       [-1.49880108e-17,  1.84174736e+00,  1.12123854e-02],
       [ 1.59500000e+00,  9.20873679e-01,  7.79471239e+00],
       [ 2.20545804e-15,  1.84174736e+00,  7.14960761e+00],
       [ 1.59500000e+00,  9.20873679e-01,  9.74410761e+00],
       [-1.59500000e+00,  4.60436840e+00,  5.20021239e+00],
       [ 3.07864845e-15,  3.68349472e+00,  2.60571239e+00],
       [-1.59500000e+00,  4.60436840e+00,  1.96060761e+00],
       [ 3.07864845e-15,  3.68349472e+00,  4.55510761e+00],
       [-1.59500000e+00,  4.60436840e+00,  1.12123854e-02],
       [ 3.07864845e-15,  3.68349472e+00,  7.79471239e+00],
       [-1.59500000e+00,  4.60436840e+00,  7.14960761e+00],
       [ 3.07864845e-15,  3.68349472e+00,  9.74410761e+00],
       [ 3.19000000e+00,  1.84174736e+00

In [28]:
g_displaced_structures = g_displacefuture.outputs.displaced_structures
g_displaced_structures

<TrajectoryData: uuid: 8ea7130c-0216-48b5-9ede-ee44324e9577 (pk: 59198)>

In [29]:
num_structures = len(g_displaced_structures.get_stepids())
num_structures


10

# load tersoff potential

In [30]:
import json
# with open("lammps_input/potentials/tersoff-GaN.json","r") as f:
with open("lammps_input/potentials/tersoff.json","r") as f:
    potential_dict = json.load(f)
lammps_potential = DataFactory("lammps.potential")(
    type=potential_dict["pair_style"], data=potential_dict["potential_dict"]
)
lammps_potential


<EmpiricalPotential: uuid: e34d14c6-238c-4456-b463-b3e0a8ede1ab (unstored)>

# execution option

In [31]:
lammps_parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': []
})
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 1}
}


# run all the displacements

In [32]:
Code.get_from_string(code_lammps_force_name.value)

<Code: Remote code 'lammps.force-lammps-localhost' on localhost, pk: 59137, uuid: 38d58d76-d1ac-419e-97e6-24a749d7c791>

In [33]:
alldisp_lammps_WorkChain = WorkflowFactory("alamode.all_displace_lammps")
# alldisp_lammps_WorkChain = alldisplaces_lammps_WorkChain


In [34]:
lammps_parameters

<Dict: uuid: 7d9234b5-8777-4ef5-8a1c-75335ef11cdf (unstored)>

In [35]:
inputs = {"structures": g_displaced_structures,
          "parameters": lammps_parameters,
          "code_string":  code_lammps_force_name,
          "cwd": g_cwd,
          "prefix": g_prefix,
          'potential': lammps_potential}



In [36]:
g_cwd

<Str: uuid: 7697e409-d2f7-4115-ab2e-63bbadf87ce6 (pk: 59122) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/Ga2N2_1x1x1_harmonic>

In [37]:
lammps_all = nodebank.load(f"{g_prefix.value}_all_lammps")
if lammps_all is None:
    lammps_all = submit(alldisp_lammps_WorkChain, **inputs)
    wait_for_node_finished(lammps_all, 5)
    print(lammps_all.is_finished_ok)
    if lammps_all.is_finished_ok:
        nodebank.dump(f"{g_prefix.value}_all_lammps", lammps_all)

wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
True True
True


In [38]:
lammps_all

<WorkChainNode: uuid: 6b0d6016-140f-491a-8f54-8ab4ecf590f0 (pk: 59203) (aiida.workflows:alamode.all_displace_lammps)>

In [39]:
g_target_folder = lammps_all.outputs.dfset_folder

In [40]:
lammps_all.outputs.dfset_folder.list_object_names()

['XFSET_harmonic01',
 'XFSET_harmonic02',
 'XFSET_harmonic03',
 'XFSET_harmonic04',
 'XFSET_harmonic05',
 'XFSET_harmonic06',
 'XFSET_harmonic07',
 'XFSET_harmonic08',
 'XFSET_harmonic09',
 'XFSET_harmonic10']

In [41]:
filename =  lammps_all.outputs.dfset_folder.list_object_names()[0]
print(lammps_all.outputs.dfset_folder.get_object_content(filename))


ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
32
ITEM: BOX BOUNDS xy xz yz pp pp pp
-3.1899999999999999e+00 6.3799999999999999e+00 -3.1899999999999999e+00
0.0000000000000000e+00 5.5252420760999996e+00 0.0000000000000000e+00
0.0000000000000000e+00 1.0378000000000000e+01 0.0000000000000000e+00
ITEM: ATOMS element xu yu zu fx fy fz 
  Ga     0.0100000000     1.8417473587     5.2002123854    -0.2708848292     0.0034007417     0.0024005341 
  Ga     1.5950000000     0.9208736794     2.6057123854     0.0002778472    -0.0008168277     0.0008174230 
   N    -0.0000000000     1.8417473587     1.9606076146    -0.0000000000     0.0000000004    -0.0000000016 
   N     1.5950000000     0.9208736794     4.5551076146     0.1133079066    -0.0538249489    -0.0380018014 
  Ga    -0.0000000000     1.8417473587     0.0112123854    -0.0000000000     0.0000000004     0.0000000016 
  Ga     1.5950000000     0.9208736794     7.7947123854     0.0002994794    -0.0008393264     0.0056057314 
   N    -0.0000000000     1

# extract

In [42]:
code_extract_name

<Str: uuid: fd25b55a-0665-4bf8-8923-26eb3542134f (unstored) value: alamode.extract@localhost>

In [43]:
g_superstructure_filename

<Str: uuid: c60d5c7e-a3c8-4b16-8707-f5fc9ed8a75b (pk: 59119) value: supercell.lammps>

In [44]:
code_extract = Code.get_from_string(code_extract_name.value)
code_extract

<Code: Remote code 'alamode.extract' on localhost, pk: 59135, uuid: 31808c39-5451-4302-9cea-a3288da8628d>

In [45]:
builder = code_extract.get_builder()
builder.format = g_format
builder.cwd = g_cwd
builder.structure_org_filename = g_superstructure_filename
builder.prefix = g_prefix
builder.target_folder = g_target_folder

In [46]:
extract = submit(builder)


In [47]:
wait_for_node_finished(extract)


wait another 2 sec.
wait another 2 sec.
True True


In [48]:
nodebank.dump(f"{g_prefix.value}_extract", extract)

<CalcJobNode: uuid: ca4871d6-94f3-494b-a004-23a6c222a36b (pk: 59272) (aiida.calculations:alamode.extract)>

In [49]:
print("done")

done


In [50]:
node = nodebank.load(f"{g_prefix.value}_extract")

In [51]:
node.outputs.dfset_file.attributes


{'filename': 'DFSET_harmonic'}

In [52]:
node.outputs.dfset_file.get_content()

"Namespace(VASP=None, QE=None, xTAPP=None, LAMMPS='supercell.lammps', OpenMX=None, get='disp-force', unitname='Rydberg', offset=None, emin=None, emax=None, target_file=['XFSET_harmonic01', 'XFSET_harmonic02', 'XFSET_harmonic03', 'XFSET_harmonic04', 'XFSET_harmonic05', 'XFSET_harmonic06', 'XFSET_harmonic07', 'XFSET_harmonic08', 'XFSET_harmonic09', 'XFSET_harmonic10'])\n# Filename: XFSET_harmonic01, Snapshot: 1\n    0.01889726124994    -0.00000000002816    -0.00000000004365      -1.05357429E-02  1.32267799E-04  9.33659155E-05\n    0.00000000000000     0.00000000008046    -0.00000000004365       1.08065360E-05 -3.17695408E-05  3.17926943E-05\n   -0.00000000000000    -0.00000000002816     0.00000000004309      -0.00000000E+00  1.55575237E-11 -6.22300949E-11\n    0.00000000000000     0.00000000008046     0.00000000004308       4.40697611E-03 -2.09345730E-03 -1.47803482E-03\n    0.00000000000000    -0.00000000002816    -0.00000000004366      -0.00000000E+00  1.55575237E-11  6.22300949E-11\n 